<a href="https://colab.research.google.com/github/aryashinod/demo/blob/main/infosecfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
Permutation-aware Confusion & Diffusion Metrics for Classical Ciphers
--------------------------------------------------------------------
- STE: symbol-transition entropy
- Spread: fraction of ciphertext positions affected
- PCS: displacement-based positional-change score
- Diffusion: if STE*Spread ≈ 0 and PCS > 0 → Diffusion = PCS (permutation-only)
             else Diffusion = (STE*Spread + PCS) / 2
- Confusion (KICE): key-induced ciphertext entropy with predictable-shift check
"""

import math, random
from collections import Counter
from statistics import mean
import pandas as pd

# ----------------------------
# Utility
# ----------------------------
ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
MOD = 26
ADFGVX = "ADFGVX"
INP = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"

def shannon_entropy(counts: Counter):
    total = sum(counts.values())
    if total == 0:
        return 0.0
    return -sum((v/total)*math.log2(v/total) for v in counts.values() if v > 0)

def normalize_entropy(entropy, alphabet_size):
    return entropy / math.log2(alphabet_size) if alphabet_size > 1 else 0.0

def random_plaintext(length, alphabet=ALPHABET):
    return ''.join(random.choice(alphabet) for _ in range(length))

# ----------------------------
# PCS helper (displacement-based)
# ----------------------------
def pcs_displacement(C, Cp):
    """Compute normalized average displacement of symbols between C and Cp."""
    n = min(len(C), len(Cp))
    if n == 0:
        return 0.0

    total_disp = 0
    for i, ch in enumerate(C[:n]):
        try:
            j = Cp.index(ch)  # position of same symbol in Cp
            total_disp += abs(j - i)
        except ValueError:
            # if symbol missing, assume max displacement
            total_disp += n
    return total_disp / (n * n)  # normalize to [0,1]

# ----------------------------
# Diffusion Metric
# ----------------------------
def diffusion_metric(encrypt, key, alphabet, plaintext, trials=200, epsilon=1e-6):
    """Permutation-aware diffusion metric with displacement-based PCS."""
    P_base = ''.join(ch for ch in plaintext.upper() if ch in alphabet)
    if len(P_base) < 5:
        P_base = random_plaintext(200, alphabet)

    N = len(alphabet)
    idx = {c: i for i, c in enumerate(alphabet)}
    delta_counts = Counter()
    pcs_list = []
    spread_list = []

    for _ in range(trials):
        pos = random.randrange(len(P_base))
        Pp = list(P_base)
        choices = [c for c in alphabet if c != P_base[pos]]
        Pp[pos] = random.choice(choices)
        Pp = ''.join(Pp)

        C = encrypt(P_base, key)
        Cp = encrypt(Pp, key)

        # STE + Spread
        changed_positions = 0
        total_positions = 0
        for a, b in zip(C, Cp):
            if a in idx and b in idx:
                delta = (idx[b] - idx[a]) % N
                delta_counts[delta] += 1
                total_positions += 1
                if a != b:
                    changed_positions += 1
        spread = (changed_positions / total_positions) if total_positions > 0 else 0.0
        spread_list.append(spread)

        # PCS via displacement
        pcs_list.append(pcs_displacement(C, Cp))

    # STE
    H = shannon_entropy(delta_counts)
    STE = normalize_entropy(H, N)

    # Spread
    spread_factor = float(mean(spread_list)) if spread_list else 0.0

    # PCS (displacement-based)
    PCS = float(mean(pcs_list)) if pcs_list else 0.0

    # Permutation-aware decision
    symbol_component = STE * spread_factor
    if (symbol_component < epsilon) and (PCS > 0.0):
        diffusion = PCS  # permutation-only diffusion
    else:
        diffusion = (symbol_component + PCS) / 2.0

    return STE, spread_factor, PCS, diffusion

# ----------------------------
# Confusion Metric (KICE)
# ----------------------------
def predictable_shift_check(ctexts, alphabet):
    N = len(alphabet)
    idx = {c:i for i,c in enumerate(alphabet)}
    base = ctexts[0]
    for c in ctexts[1:]:
        deltas = set()
        for a,b in zip(base,c):
            if a in idx and b in idx:
                deltas.add((idx[b]-idx[a])%N)
        if len(deltas) != 1:
            return False
    return True

def key_induced_ciphertext_entropy(encrypt, key, key_neighbours, alphabet,
                                   plaintext, max_neigh=200):
    P = ''.join(ch for ch in plaintext.upper() if ch in alphabet)
    if len(P) < 5:
        P = random_plaintext(200, alphabet)
    N = len(alphabet)
    idx = {c:i for i,c in enumerate(alphabet)}
    ctexts = [encrypt(P, key)]
    for j,kn in enumerate(key_neighbours(key)):
        if j >= max_neigh:
            break
        ctexts.append(encrypt(P, kn))

    # If predictable uniform shift across neighbours -> confusion = 0
    if predictable_shift_check(ctexts, alphabet):
        return 0.0

    minlen = min(len(c) for c in ctexts)
    pos_counters = [Counter() for _ in range(minlen)]
    for c in ctexts:
        for i,ch in enumerate(c[:minlen]):
            if ch in idx:
                pos_counters[i][ch]+=1
    entropies = [normalize_entropy(shannon_entropy(cnt), N) for cnt in pos_counters]
    return float(mean(entropies)) if entropies else 0.0

# ----------------------------
# Cipher Implementations
# ----------------------------
def encrypt_shift(p,k):
    return ''.join(ALPHABET[(ALPHABET.index(ch)+k)%MOD] if ch in ALPHABET else ch for ch in p.upper())

def shift_neigh(k):
    yield (k+1)%MOD
    yield (k-1)%MOD

def encrypt_subst(p,keymap):
    mp = {ALPHABET[i]:keymap[i] for i in range(26)}
    return ''.join(mp.get(ch,ch) for ch in p.upper())

def subst_neigh(keymap):
    key = list(keymap)
    for _ in range(200):
        a,b = random.sample(range(26),2)
        k2 = key.copy(); k2[a], k2[b] = k2[b], k2[a]
        yield ''.join(k2)

def encrypt_vig(p,key):
    out=[]; key = key.upper(); j = 0
    for ch in p.upper():
        if ch in ALPHABET:
            k = ALPHABET.index(key[j % len(key)])
            out.append(ALPHABET[(ALPHABET.index(ch) + k) % MOD])
            j += 1
        else:
            out.append(ch)
    return ''.join(out)

def vig_neigh(key):
    for i in range(len(key)):
        base = ALPHABET.index(key[i])
        for d in (-1,1):
            k = list(key); k[i] = ALPHABET[(base + d) % MOD]; yield ''.join(k)

def encrypt_hill3(p,matrix):
    txt=''.join(ch for ch in p.upper() if ch in ALPHABET)
    if len(txt) % 3:
        txt += 'X' * (3 - (len(txt) % 3))
    out=[]
    for i in range(0, len(txt), 3):
        vec = [ALPHABET.index(txt[i+j]) for j in range(3)]
        res = [sum(matrix[r*3+c] * vec[c] for c in range(3)) % 26 for r in range(3)]
        out.extend(ALPHABET[x] for x in res)
    return ''.join(out)

def hill_neigh(matrix):
    for pos in range(9):
        for d in (-1,1):
            m = matrix.copy(); m[pos] = (m[pos] + d) % 26; yield m

def encrypt_trans(p,key):
    txt=''.join(ch for ch in p.upper() if ch in ALPHABET)
    cols = len(key); rows = (len(txt) + cols - 1) // cols
    grid = [['X'] * cols for _ in range(rows)]
    k=0
    for r in range(rows):
        for c in range(cols):
            if k < len(txt):
                grid[r][c] = txt[k]
            k += 1
    out=[]
    for col in key:
        for r in range(rows): out.append(grid[r][col])
    return ''.join(out)

def trans_neigh(key):
    for i in range(len(key)-1):
        k = key.copy(); k[i], k[i+1] = k[i+1], k[i]; yield k

def polybius_map(phrase):
    seen=[]
    for ch in phrase.upper() + INP:
        if ch in INP and ch not in seen: seen.append(ch)
    grid = [seen[i:i+6] for i in range(0,36,6)]
    return { grid[r][c] : ADFGVX[r] + ADFGVX[c] for r in range(6) for c in range(6) }

def encrypt_adfgvx(p,key):
    phrase, trans = key
    mp = polybius_map(phrase)
    txt=''.join(ch for ch in p.upper() if ch in INP)
    pairs = ''.join(mp[ch] for ch in txt)
    cols = len(trans); rows = (len(pairs) + cols - 1) // cols
    grid = [['X'] * cols for _ in range(rows)]
    k=0
    for r in range(rows):
        for c in range(cols):
            if k < len(pairs):
                grid[r][c] = pairs[k]
            k += 1
    out=[]
    for col in trans:
        for r in range(rows): out.append(grid[r][col])
    return ''.join(out)

def adfgvx_neigh(key):
    phrase, trans = key
    for i in range(len(trans)-1):
        t = trans.copy(); t[i], t[i+1] = t[i+1], t[i]; yield (phrase, t)
    for _ in range(50):
        letters = list(INP); random.shuffle(letters)
        yield (''.join(letters), trans)

# ----------------------------
# Run Experiment
# ----------------------------
if __name__ == "__main__":
    random.seed()
    PLAINTEXT_LENGTH = 400
    TRIALS = 200

    user_plaintext = random_plaintext(PLAINTEXT_LENGTH, ALPHABET)

    results = []

    # Shift
    ste, spread, pcs, diff = diffusion_metric(encrypt_shift, 3, ALPHABET, user_plaintext, trials=TRIALS)
    results.append(("Shift Cipher", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_shift, 3, shift_neigh, ALPHABET, user_plaintext)))

    # Substitution
    letters = list(ALPHABET); random.shuffle(letters); keymap = ''.join(letters)
    ste, spread, pcs, diff = diffusion_metric(encrypt_subst, keymap, ALPHABET, user_plaintext, trials=TRIALS)
    results.append(("Substitution Cipher", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_subst, keymap, subst_neigh, ALPHABET, user_plaintext)))

    # Vigenere
    vigkey = "MUSIC"
    ste, spread, pcs, diff = diffusion_metric(encrypt_vig, vigkey, ALPHABET, user_plaintext, trials=TRIALS)
    results.append(("Vigenere Cipher", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_vig, vigkey, vig_neigh, ALPHABET, user_plaintext)))

    # Hill
    hillkey = [2,4,5,9,2,1,3,17,7]
    ste, spread, pcs, diff = diffusion_metric(encrypt_hill3, hillkey, ALPHABET, user_plaintext, trials=TRIALS)
    results.append(("Hill Cipher (3x3)", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_hill3, hillkey, hill_neigh, ALPHABET, user_plaintext)))

    # Transposition
    transkey = [2,0,5,1,4,3]
    ste, spread, pcs, diff = diffusion_metric(encrypt_trans, transkey, ALPHABET, user_plaintext, trials=TRIALS)
    results.append(("Transposition Cipher", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_trans, transkey, trans_neigh, ALPHABET, user_plaintext)))

    # ADFGVX
    adfkey = ("SECRET", [5,2,0,4,1,3])
    ste, spread, pcs, diff = diffusion_metric(encrypt_adfgvx, adfkey, ADFGVX, user_plaintext, trials=TRIALS)
    results.append(("ADFGVX Cipher", ste, spread, pcs, diff,
                    key_induced_ciphertext_entropy(encrypt_adfgvx, adfkey, adfgvx_neigh, ADFGVX, user_plaintext)))

    df = pd.DataFrame(results, columns=["Cipher","STE","Spread","PCS","Diffusion","KICE (Confusion)"])
    df['Average Score'] = (df['Diffusion'] + df['KICE (Confusion)']) / 2.0
    df = df.sort_values(by='Average Score', ascending=False).reset_index(drop=True)

    print("\n--- Final Confusion and Diffusion Analysis ---")
    print("Plaintext length:", PLAINTEXT_LENGTH, "| Trials:", TRIALS)
    print("-" * 80)
    print(df.to_string(index=False))



--- Final Confusion and Diffusion Analysis ---
Plaintext length: 400 | Trials: 200
--------------------------------------------------------------------------------
              Cipher      STE   Spread      PCS  Diffusion  KICE (Confusion)  Average Score
       ADFGVX Cipher 0.035650 0.008778 0.468162   0.234237          0.953522       0.593880
   Hill Cipher (3x3) 0.020459 0.007351 0.439981   0.220066          0.336727       0.278396
Transposition Cipher 0.007760 0.002488 0.449135   0.224577          0.210753       0.217665
     Vigenere Cipher 0.007789 0.002500 0.428548   0.214284          0.184208       0.199246
 Substitution Cipher 0.007758 0.002500 0.429170   0.214595          0.136980       0.175787
        Shift Cipher 0.007779 0.002500 0.429145   0.214582          0.000000       0.107291
